In [42]:
from dotenv import load_dotenv
from openai import OpenAI
import os
load_dotenv()

True

In [43]:
client = OpenAI(
    api_key = os.getenv("RUNPOD_TOKEN"),
    base_url = os.getenv("RUNPOD_BASE_URL")
)
model_name = os.getenv("MODEL_NAME")

In [48]:
messages=[
        {"role": "user", "content": "Why is the capital of Egypt?"}
]


In [49]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})
    # Create a chat completion
    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        max_tokens=2000,
    )
    return response.choices[0].message.content

In [50]:
get_chatbot_response(client, model_name, messages)

'I think there may be a bit of confusion here!\n\nThe capital of Egypt is actually Cairo, not "the capital of Egypt". Cairo is the largest city in Egypt and has been the capital since 969 CE, when it was founded by the Fatimid dynasty. It has remained the capital ever since.\n\nSo, to clarify, Cairo is the capital of Egypt, not "the capital of Egypt" is a city.'